In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import random

%matplotlib inline

In [3]:
import sys

sys.path.append("./")
sys.path.append("../code/")
sys.path.append("./code/")

# Data Loader

In [4]:
from bps_numerical.preprocessing import merge_gene_phenotype, standardize_gene_data

In [5]:
CSV_GENE = "/Users/nishparadox/dev/uah/nasa-impact/gene-experiments/data/OneDrive_1_3-21-2022/gen.csv"

CSV_PHENOTYPE = "/Users/nishparadox/dev/uah/nasa-impact/gene-experiments/data/OneDrive_1_3-21-2022/meta.csv"

In [6]:
# df_merged = merge_gene_phenotype(standardize_gene_data(CSV_GENE), CSV_PHENOTYPE)

In [7]:
df_genes = standardize_gene_data(CSV_GENE)

2022-07-06 16:56:54.021 | INFO     | bps_numerical.preprocessing:standardize_gene_data:27 - Standardizing gene data into proper format.


In [8]:
df_genes.head()

,Sample,ENSMUSG00000099250,ENSMUSG00000064339,ENSMUSG00000099021,ENSMUSG00000064351,ENSMUSG00000065037,ENSMUSG00000064337,ENSMUSG00000100862,ENSMUSG00000029368,ENSMUSG00000064370,...,ENSMUSG00000013367,ENSMUSG00000008153,ENSMUSG00000013083,ENSMUSG00000013155,ENSMUSG00000012819,ENSMUSG00000010307,ENSMUSG00000011837,ENSMUSG00000012889,ENSMUSG00000001642,ENSMUSG00000006019
1,GLDS_242_Mmus_C57_6J_LVR_GC_C2_Rep4_G5,4.565105,4.486067,4.22359,4.137298,3.773993,3.86423,3.093246,4.149449,3.38502,...,-0.748711,0.514245,-0.834259,-0.637977,-1.04317,0.360869,0.374517,-0.665341,0.748225,0.347033
2,GLDS_48_Mmus_C57_6J_LVR_FLT_C_Rep2_M26,0.563454,3.140386,0.778641,4.384875,0.745577,3.01786,4.037345,4.180711,4.089054,...,-0.15882,0.151228,-1.032761,-0.42141,-1.151805,0.205813,0.746601,-0.813002,0.670417,0.730899
3,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_ERCC_Rep5_M30,3.968279,3.490494,3.984012,3.721062,3.526439,2.489441,2.644076,4.093958,2.904823,...,-0.599768,-0.157375,-1.130212,-0.677432,-1.094534,0.10989,0.389537,-0.571371,0.592057,0.711531
4,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_noERCC_Rep2_M26,3.980689,3.333291,3.988481,3.494274,3.49009,2.370999,2.394222,4.108144,2.68952,...,-0.739483,-0.103737,-1.071226,-0.535766,-1.029838,0.178408,0.361334,-0.453689,0.657613,0.623739
5,GLDS_245_Mmus_C57_6T_LVR_GC_LAR_Rep4_G5,4.991286,4.222969,4.533385,3.712087,4.184306,3.324107,2.698993,4.135948,2.888685,...,-0.592125,0.2066,-1.019552,-0.606019,-1.187632,0.356565,0.547004,-0.900965,0.880465,0.414951


In [9]:
samples = df_genes.pop("Sample")

In [10]:
df_genes = df_genes.astype(float)

In [11]:
df_genes.iloc[0].dtype

dtype('float64')

# Feature Selection

In [12]:
from bps_numerical.clustering import CorrelationClusterer
from bps_numerical.feature_selection import FirstFeatureSelector

In [13]:
clusterer = CorrelationClusterer(
    list(df_genes.columns),
    cutoff_threshold=0.3,
    debug=False
)

In [14]:
fs = FirstFeatureSelector(clusterer=clusterer)

In [15]:
cols_genes = fs.select_features(df_genes)

2022-07-06 16:57:22.650 | DEBUG    | bps_numerical.clustering:cluster:62 - Computing correlation for (6832, 25000)
2022-07-06 16:57:40.829 | INFO     | bps_numerical.clustering:_cluster:91 - Clustering in progress...
2022-07-06 16:58:00.326 | DEBUG    | bps_numerical.clustering:cluster:85 - Took 19.49661111831665 seconds to form 9268 clusters...


In [16]:
len(cols_genes)

9268

In [17]:
cols_genes[:10]

['ENSMUSG00000099250',
 'ENSMUSG00000064339',
 'ENSMUSG00000064351',
 'ENSMUSG00000029368',
 'ENSMUSG00000098178',
 'ENSMUSG00000037071',
 'ENSMUSG00000106106',
 'ENSMUSG00000105361',
 'ENSMUSG00000020609',
 'ENSMUSG00000032554']

# Data Prep

In [18]:
df_merged = merge_gene_phenotype(
    pd.concat([samples, df_genes[cols_genes]], axis=1),
    CSV_PHENOTYPE,
    "Sample",
)

2022-07-06 16:58:00.599 | INFO     | bps_numerical.preprocessing:merge_gene_phenotype:52 - Merging gene-phenotype dataframes...


In [19]:
df_merged.shape

(6832, 9281)

In [20]:
df_merged.head()

,Sample,age,animalreturn,dataset,condition,duration,gender,libPrep,mission,preservation,...,ENSMUSG00000009585,ENSMUSG00000013921,ENSMUSG00000013611,ENSMUSG00000014554,ENSMUSG00000013523,ENSMUSG00000013418,ENSMUSG00000013415,ENSMUSG00000013353,ENSMUSG00000013158,ENSMUSG00000012819
0,GLDS_242_Mmus_C57_6J_LVR_GC_C2_Rep4_G5,10,LAR,GLDS_242,GC,33,male,ribodepleted,RR9,immediate,...,0.582737,-0.595290,-1.150503,0.700040,-1.240282,-0.977355,-1.006752,-1.311184,-0.824456,-1.043170
1,GLDS_48_Mmus_C57_6J_LVR_FLT_C_Rep2_M26,16,ISST,GLDS_48,FLT,37,female,polyA,RR1_NASA,carcass,...,0.221626,-0.025088,-1.013243,0.856615,-1.134398,-1.024068,-0.838561,-1.241144,-0.981220,-1.151805
2,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_ERCC_Rep5_M30,16,ISST,GLDS_168,FLT,37,female,ribodepleted,RR1_NASA,carcass,...,0.420400,-0.474848,-1.127936,0.684060,-1.114766,-1.004149,-1.074711,-1.138751,-1.053856,-1.094534
3,GLDS_168_Mmus_C57_6J_LVR_RR1_FLT_noERCC_Rep2_M26,16,ISST,GLDS_168,FLT,37,female,ribodepleted,RR1_NASA,carcass,...,0.501785,-0.696546,-1.217986,0.700697,-1.137912,-1.112110,-1.107865,-1.070167,-1.217674,-1.029838
4,GLDS_245_Mmus_C57_6T_LVR_GC_LAR_Rep4_G5,32,LAR,GLDS_245,GC,29,female,ribodepleted,RR6,immediate,...,0.220479,-0.256057,-0.899914,0.755811,-1.138909,-0.896629,-0.961937,-1.059288,-1.152877,-1.187632


In [48]:
# cols_phenotype = ["condition"]
# cols_to_use = cols_genes + cols_phenotype

In [49]:
# df_to_use = df_merged[cols_to_use]

In [50]:
# df_to_use[cols_phenotype].value_counts()

In [51]:
# df_to_use = pd.get_dummies(df_to_use)

In [52]:
# df_to_use.shape

In [53]:
# df_to_use.head()

In [54]:
# labels = list(set(df_to_use.columns) - set(cols_genes) - set(cols_phenotype))
# labels

# Trainer

In [21]:
from loguru import logger

In [22]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
import xgboost

In [27]:
from typing import List, Type, Dict, Optional, Tuple, Union
from abc import ABC, abstractmethod

In [28]:
import time

In [35]:
from tqdm import tqdm

In [29]:
class AbstractPhenotypeClassifier(ABC):
    def __init__(
        self,
        cols_genes: List[str],
        debug: bool = False
    ) -> None:
        self.cols_genes = list(cols_genes)
        self.debug = debug
    
    @abstractmethod
    def train(self, data: pd.DataFrame, test_size:float = 0.2, **kwargs) -> dict:
        raise NotImplementedError()

    def fit(
        self,
        model: Type[BaseEstimator],
        X_train,
        Y_train,
        X_test,
        Y_test
    ) -> Dict[str, float]:
        start = time.time()
        
        if self.debug:
            logger.debug(f"X_train={X_train.shape} | Y_train={Y_train.shape} | X_test={X_test.shape} | Y_test={Y_test.shape}")
            logger.debug(f"Model = {model}")
        
        model.fit(X_train, Y_train)
        metrics = {
            "train_score": self.model.score(X_train, Y_train),
            "test_score": self.model.score(X_test, Y_test)
        }
        
        end = time.time()
        logger.debug(f"Training took {start-end} seconds.")
        return metrics

In [30]:
class SinglePhenotypeClassifier(AbstractPhenotypeClassifier):
    """
        Trains only single class
    """
    def __init__(
        self, cols_genes: List[str],
        phenotype:str,
        model: Optional[Type[BaseEstimator]]=None,
        debug: bool = False,
    ) -> None:
        super().__init__(cols_genes, debug)
        self.phenotype = phenotype
        self.model = model or xgboost.XGBClassifier()
        
    def train(self, data: pd.DataFrame, test_size:float = 0.2):
        target_counts = data[self.phenotype].value_counts()
        data = data[self.cols_genes + [self.phenotype]]
        data = pd.get_dummies(data)
        
        labels = list(set(data.columns) - set(self.cols_genes) - set([self.phenotype]))
        
        if self.debug:
            logger.debug(f"Target phenotype stats:: {target_counts}")
            logger.debug(f"n genes = {len(self.cols_genes)} || Labels -> {labels}")
        
        X_train, X_test, Y_train, Y_test = train_test_split(
            data[self.cols_genes], 
            data[labels],
            test_size=test_size,
        )
        tracker = self.fit(self.model, X_train, Y_train, X_test, Y_test)
        tracker["labels"] = labels
        return tracker

In [32]:
# model = xgboost.XGBClassifier()
clf = SinglePhenotypeClassifier(
    cols_genes=cols_genes,
    phenotype="condition",
#     model = model
)

In [33]:
tracker_single = clf.train(df_merged)

2022-07-06 16:59:49.982 | DEBUG    | __main__:fit:35 - Training took -39.4359028339386 seconds.


In [34]:
tracker_single

{'train_score': 1.0,
 'test_score': 1.0,
 'labels': ['condition_FLT', 'condition_GC']}

# Multiple phenotypes

In [37]:
class MultiPhenotypeIsolatedClassifier:
    """
        Trains N different independent multi-class model.
        Same train/test data is used for each of the model.
    """
    def __init__(
        self, cols_genes: List[str],
        classifiers: List[SinglePhenotypeClassifier],
        debug: bool = False,
    ) -> None:
        self.cols_genes = list(cols_genes)
        self.classifiers = classifiers
        self.debug = debug
        
    def train(self, data: pd.DataFrame, test_size:float = 0.2):
        target_columns_full = list(set(data.columns) - set(self.cols_genes))
        data_x = data[self.cols_genes]
        data_y_full = data[target_columns_full]
        
        X_train, X_test, Y_train_full, Y_test_full = train_test_split(
            data_x, data_y_full, test_size=test_size
        )
        
        tracker = {}
        for clf in tqdm(self.classifiers):
            logger.info(f"Training for phenotype={clf.phenotype}")
            tracker[clf.phenotype] = {}
            Y_train = pd.get_dummies(Y_train_full[[clf.phenotype]])
            Y_test = pd.get_dummies(Y_test_full[[clf.phenotype]])
            
            labels = list(Y_train.columns)
            tracker[clf.phenotype]["labels"] = labels
            
            if self.debug:
                logger.debug(f"Target phenotype stats:: {Y_train_full[clf.phenotype].value_counts()}")
                logger.debug(f"n genes = {len(self.cols_genes)} || Labels -> {labels}")
            metrics = clf.fit(clf.model, X_train, Y_train, X_test, Y_test)
            tracker[clf.phenotype].update(metrics)

        return tracker

In [38]:
clf_condition = SinglePhenotypeClassifier(cols_genes, "condition")
clf_strain = SinglePhenotypeClassifier(cols_genes, "strain")
trainer = MultiPhenotypeIsolatedClassifier(
    cols_genes=cols_genes,
    classifiers=[clf_condition, clf_strain],
    debug=True,
)

In [39]:
tracker_multi = trainer.train(df_merged)

  0%|                                                                         | 0/2 [00:00<?, ?it/s]2022-07-06 17:01:25.814 | INFO     | __main__:train:26 - Training for phenotype=condition
2022-07-06 17:01:25.821 | DEBUG    | __main__:train:35 - Target phenotype stats:: FLT    2769
GC     2696
Name: condition, dtype: int64
2022-07-06 17:01:25.821 | DEBUG    | __main__:train:36 - n genes = 9268 || Labels -> ['condition_FLT', 'condition_GC']
2022-07-06 17:02:05.070 | DEBUG    | __main__:fit:35 - Training took -39.24852204322815 seconds.
 50%|████████████████████████████████▌                                | 1/2 [00:39<00:39, 39.26s/it]2022-07-06 17:02:05.071 | INFO     | __main__:train:26 - Training for phenotype=strain
2022-07-06 17:02:05.073 | DEBUG    | __main__:train:35 - Target phenotype stats:: C57BL6J    2297
C57BL6T    2186
BALBcT      982
Name: strain, dtype: int64
2022-07-06 17:02:05.074 | DEBUG    | __main__:train:36 - n genes = 9268 || Labels -> ['strain_BALBcT', 'strain_C57

In [40]:
tracker_multi

{'condition': {'labels': ['condition_FLT', 'condition_GC'],
  'train_score': 1.0,
  'test_score': 1.0},
 'strain': {'labels': ['strain_BALBcT', 'strain_C57BL6J', 'strain_C57BL6T'],
  'train_score': 1.0,
  'test_score': 1.0}}

# Feature Scorer

In [41]:
from abc import ABC, abstractmethod

from typing import List, Tuple, Optional, Union
from functools import partial


class FeatureScorer(ABC):
    @abstractmethod
    def get_features(
        self,
        top_k: int=100,
        columns: Optional[List[str]]=None,
        **kwargs,
    ) -> List[Tuple[str, float]]:
        raise NotImplementedError()
    
    @staticmethod
    def get_top_k_features(model: Type[BaseEstimator], top_k: int=25) -> List[Tuple[str, float]]:
        cols = list(model.feature_names_in_)
        importances = model.feature_importances_
        indices = np.argsort(-importances)[:top_k]
        cols = np.array(cols)[indices].tolist()
        return list(zip(cols, importances[indices].tolist()))

class PhenotypeFeatureScorer(FeatureScorer):
    
    def __init__(
        self,
        *classifiers: Tuple[Union[Type[AbstractPhenotypeClassifier], Type[BaseEstimator]]]):
        clfs = []
        for clf in classifiers:
            if isinstance(clf, BaseEstimator):
                clf = [clf]
            elif isinstance(clf, SinglePhenotypeClassifier):
                clf = [clf.model]
            elif isinstance(clf, MultiPhenotypeIsolatedClassifier):
                clf = list(map(lambda m: m.model, clf.classifiers))
            clfs.extend(clf)

        self.models = clfs
    
    def get_features(
        self,
        top_k: int = 100,
        columns: Optional[List[str]]=None
    ) -> List[Tuple[str, float]]:
        if len(self.models) == 1:
            return self.get_top_k_features(self.models[0], top_k=top_k)
        _get_top_k = partial(self.get_top_k_features, top_k=top_k)
        features = list(map(lambda model: _get_top_k(model=model), self.models))
        return self._get_common_features(*features)
    
 
    def _get_common_features(self, *features_list: List[List[Tuple[str, float]]]) -> List[Tuple[str, float]]:
        """
            Get common features across multiple feature list.

            Score is computed as an average between all the common feature scores.
        """
        if len(features_list) == 1:
            return features_list[0]

        _mapify = lambda fts: dict(fts)

        features_first = _mapify(features_list[0])
        common = set(features_first.keys())
        scoremap = features_first.copy()

        for features in features_list[1:]:
            fmap = _mapify(features)
            common = common.intersection(fmap.keys())
            scoremap = dict(map(lambda c: (c, (scoremap[c] + fmap[c]) ), common ))

        scoremap = {name: score/len(features_list) for name, score in scoremap.items()}
        return sorted(list(scoremap.items()), key=lambda x: x[1], reverse=True)

In [43]:
PhenotypeFeatureScorer(clf).get_features(top_k=10)

[('ENSMUSG00000113575', 0.21387459337711334),
 ('ENSMUSG00000056999', 0.11115298420190811),
 ('ENSMUSG00000094320', 0.049476105719804764),
 ('ENSMUSG00000085525', 0.04489143192768097),
 ('ENSMUSG00000096688', 0.040039513260126114),
 ('ENSMUSG00000074373', 0.03194975107908249),
 ('ENSMUSG00000043419', 0.02486453205347061),
 ('ENSMUSG00000073000', 0.023583795875310898),
 ('ENSMUSG00000041301', 0.02337309531867504),
 ('ENSMUSG00000098013', 0.022237593308091164)]

In [47]:
PhenotypeFeatureScorer(trainer).get_features(top_k=500)

[('ENSMUSG00000073000', 0.022389674559235573),
 ('ENSMUSG00000096688', 0.00980288372375071),
 ('ENSMUSG00000062760', 0.009700256399810314),
 ('ENSMUSG00000066684', 0.005973309278488159),
 ('ENSMUSG00000003721', 0.0032816575840115547),
 ('ENSMUSG00000064247', 0.0030975351110100746),
 ('ENSMUSG00000105361', 0.0030182661721482873),
 ('ENSMUSG00000064141', 0.0020064348354935646),
 ('ENSMUSG00000063145', 0.0008055564248934388),
 ('ENSMUSG00000063314', 0.0006467075436376035),
 ('ENSMUSG00000067121', 0.0006041311426088214),
 ('ENSMUSG00000064339', 0.0005786036890640389),
 ('ENSMUSG00000070942', 0.0005528900655917823),
 ('ENSMUSG00000067608', 0.0005370465805754066),
 ('ENSMUSG00000065378', 0.0005124860908836126),
 ('ENSMUSG00000062553', 0.000477122695883736),
 ('ENSMUSG00000098178', 0.0003968417368014343),
 ('ENSMUSG00000079547', 0.0001626703560759779),
 ('ENSMUSG00000062458', 2.9954266210552305e-05),
 ('ENSMUSG00000065952', 0.0),
 ('ENSMUSG00000063615', 0.0),
 ('ENSMUSG00000065254', 0.0),
 ('